# Day 23 - Linked list reordering

* https://adventofcode.com/2020/day/23

I strongly suspect that part 2 will ask us to turn the knobs up on the number of rounds and number of cups, but for part 1 I just implemented the described steps.

My initial implementation used a standard Python list that rebuilt each round. But, I then remembered how I recently adviced a client to not use an 'ordering' column in a SQL table but instead [simulate a linked list](https://www.sqlservercentral.com/articles/linked-lists-1), to minimise what rows have to be updated each time a row changes its location. So I switched to doing the same for the cups:

- Each cup _label_ becomes an index in to a list.
- Each list _value_ becomes the index of the cup that follows (singly linked list)

Reordering then becomes a simple assignment operation to the 'current' cup and the preceding index in the list (adjusted to loop around as needed, skipping affected removed cups):

- The current cup index is updated to reference the cup after the 3 removed.
- The preceding cup is updated to reference what the current cup used to reference.
- The third removed cup is updated to referece what the preceding cup used to reference.

Because the labels start at 1, index 0 is kept at `None`.

In [1]:
from dataclasses import dataclass
from typing import List

class CupGame:
    cups: List[int]
    
    def __init__(self, cuplabels: str) -> None:
        labels = [int(c) for c in cuplabels]
        cups = self.cups = [None] * (max(labels) + 1)
        prev = labels[-1]
        for label in labels:
            cups[prev] = label
            prev = label
        self._current = labels[0]

    def play_rounds(self, rounds: int = 100) -> "CupGame":
        cups, current, l = self.cups, self._current, len(self.cups)
        for _ in range(rounds):
            target = current - 1 if current > 1 else l - 1
            removed = current
            removed_labels = [removed := cups[removed] for i in range(3)]
            while target in removed_labels:
                target = target - 1 if target > 1 else l - 1
            cups[current], cups[target], cups[removed] = cups[removed], cups[current], cups[target]
            self._current = current = cups[current]
        return self
    
    @property
    def follows_one(self):
        cups, label, count = self.cups, 1, len(self.cups) - 2
        return ''.join([str(label := cups[label]) for _ in range(count)])

assert CupGame('389125467').play_rounds(10).follows_one == '92658374'
assert CupGame('389125467').play_rounds().follows_one == '67384529'

In [2]:
import aocd
cupdata = aocd.get_data(day=23, year=2020)

In [3]:
print("Part 1:", CupGame(cupdata).play_rounds().follows_one)

Part 1: 27865934
